In [114]:
import yfinance as yf

# 定義股票代碼
tickers = ['2302.TW', '2303.TW', '2329.TW', '2330.TW', '2337.TW', '2338.TW', '2340.TW', 
          '2342.TW', '2344.TW', '2351.TW', '2363.TW', '2369.TW', '2379.TW', '2388.TW', 
          '2401.TW', '2408.TW', '2434.TW', '2436.TW', '2441.TW', '2449.TW', '2451.TW', 
          '2454.TW', '2458.TW', '2481.TW', '3006.TW', '3014.TW', '3016.TW', '3034.TW', 
          '3035.TW', '3041.TW']

# 下載數據
data = yf.download(tickers, start='2024-01-01', end='2024-12-01')

# 保存收盤價資料
closing_prices = data['Adj Close'].dropna(how='all', axis=1)  # 清除全為NaN的列


[*********************100%***********************]  30 of 30 completed


In [115]:
import numpy as np

def initialize_population(num_portfolios, num_stocks):
    population = np.random.rand(num_portfolios, num_stocks)  # 直接生成浮点数
    population /= population.sum(axis=1)[:, None]  # 按行标准化，保证权重总和为1
    for i in range(num_portfolios):
        weights = np.random.rand(num_stocks)  # 生成随机权重
        weights /= np.sum(weights)  # 确保权重总和为1
        population[i, :] = weights
    return population


# 初始化族群
num_portfolios = 50
population = initialize_population(num_portfolios, len(closing_prices.columns))


In [116]:
def portfolio_performance(weights, returns):
    return np.sum(returns.mean() * weights) * 252  # 年化回報率

def evaluate_population(population, returns):
    performances = []
    for i in range(len(population)):
        performances.append(portfolio_performance(population[i], returns))
    return performances

# 計算初始族群的表現
daily_returns = closing_prices.pct_change()  # 計算日收益率
fitness_scores = evaluate_population(population, daily_returns)


In [117]:
def normalize_fitness_scores(scores):
    min_score = min(scores)
    if min_score < 0:
        scores = [score + abs(min_score) + 0.01 for score in scores]  # 确保最小值为正数
    total = sum(scores)
    probabilities = [score / total for score in scores]
    return probabilities

def select(population, fitness_scores):
    # 基於適應度分數進行選擇
    probabilities = fitness_scores / np.sum(fitness_scores)
    probabilities = normalize_fitness_scores(probabilities)
    indices = np.random.choice(np.arange(len(population)), size=len(population), replace=True, p=probabilities)
    return population[indices]


In [118]:
def crossover_and_mutation(population, crossover_rate=0.8, mutation_rate=0.05):
    # 交叉
    new_population = []
    for i in range(0, len(population), 2):
        parent1, parent2 = population[i], population[(i+1) % len(population)]
        if np.random.rand() < crossover_rate:
            crossover_point = np.random.randint(1, len(parent1))
            child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
            
            child1 /= np.sum(child1)  # 确保权重总和为1
            child2 /= np.sum(child2)
        else:
            child1, child2 = parent1, parent2
        new_population.extend([child1, child2])
    
    # 变异
    for i in range(len(new_population)):
        if np.random.rand() < mutation_rate:
            mutation_point = np.random.randint(len(new_population[i]))
            mutation_value = new_population[i][mutation_point] * (1 + np.random.normal(0, 0.1))  # 增加小的正态扰动
            new_population[i][mutation_point] = mutation_value if mutation_value > 0 else 0  # 确保新权重为正
            new_population[i] /= np.sum(new_population[i])  # 确保权重总和为1
    
    return np.array(new_population)

# 进行一代遗传操作
selected_population = select(population, fitness_scores)
print(selected_population)
new_population = crossover_and_mutation(selected_population)


[[0.04492874 0.05108043 0.03986819 ... 0.05959877 0.05797188 0.05249039]
 [0.03958993 0.02748846 0.01341506 ... 0.05856476 0.0609898  0.02366102]
 [0.03388586 0.04764059 0.02857607 ... 0.05209699 0.00055831 0.05300671]
 ...
 [0.01088215 0.06129743 0.01863819 ... 0.02530747 0.02869403 0.05275557]
 [0.05302398 0.03110556 0.0101385  ... 0.04136581 0.00534982 0.02741646]
 [0.02807137 0.04757924 0.05181921 ... 0.04617428 0.0520048  0.00605864]]


In [119]:
def print_selected_stocks(portfolio, stock_names):
    selected_stocks = [stock_names[i] for i in range(len(portfolio)) if portfolio[i] == 1]
    print("選中的個股:", ', '.join(selected_stocks))

# 在迭代過程中添加顯示選中的個股
num_generations = 100  # 設定迭代的代數
best_portfolio_overall = None
best_fitness_overall = float('-inf')
stock_names = closing_prices.columns  # 股票名稱

for generation in range(num_generations):
    # 選擇
    probabilities = normalize_fitness_scores(fitness_scores)
    selected_population = select(population, probabilities)

    # 交叉和突變
    new_population = crossover_and_mutation(selected_population)

    # 評估新族群的表現
    fitness_scores = evaluate_population(new_population, daily_returns)

    # 更新族群
    population = new_population

    # 檢查並保存最佳表現
    current_best_fitness = max(fitness_scores)
    current_best_index = fitness_scores.index(current_best_fitness)
    if current_best_fitness > best_fitness_overall:
        best_fitness_overall = current_best_fitness
        best_portfolio_overall = population[current_best_index]

    # 打印當前代的信息和選中的個股
    if generation % 10 == 0:
        print(f"Generation {generation}: Best fitness {best_fitness_overall}")
        print_selected_stocks(population[current_best_index], stock_names)

print("Optimization completed.")


Generation 0: Best fitness -0.004491474303415032
選中的個股: 
Generation 10: Best fitness 0.07821621593631554
選中的個股: 
Generation 20: Best fitness 0.10205824430607928
選中的個股: 
Generation 30: Best fitness 0.10848856168734983
選中的個股: 
Generation 40: Best fitness 0.12055359345690786
選中的個股: 
Generation 50: Best fitness 0.1262237647730751
選中的個股: 
Generation 60: Best fitness 0.1275757443055111
選中的個股: 
Generation 70: Best fitness 0.1275757443055111
選中的個股: 
Generation 80: Best fitness 0.1321797591926446
選中的個股: 
Generation 90: Best fitness 0.1321797591926446
選中的個股: 
Optimization completed.


In [120]:
# 最佳投資組合的回報率和風險計算
best_return = np.sum(daily_returns.mean() * best_portfolio_overall) * 252  # 年化回報率
best_risk = np.sqrt(np.dot(best_portfolio_overall.T, np.dot(daily_returns.cov() * 252, best_portfolio_overall)))  # 年化風險

print("最佳投資組合權重:", {
    stock_names[i]: best_portfolio_overall[i] for i in range(len(best_portfolio_overall))
})
print("最佳投資組合的年化回報率:", best_return)
print("最佳投資組合的年化風險:", best_risk)


最佳投資組合權重: {'2302.TW': 0.08549561997731864, '2303.TW': 0.04926064862972103, '2329.TW': 0.01335302448085793, '2330.TW': 0.10194121871639523, '2337.TW': 0.0020299789777812514, '2338.TW': 0.008175360958323913, '2340.TW': 0.0040250038445489985, '2342.TW': 0.09206719084730794, '2344.TW': 0.012450133763008069, '2351.TW': 0.02264681556922204, '2363.TW': 0.07186750707586737, '2369.TW': 0.0586594428297497, '2379.TW': 0.033043985849950046, '2388.TW': 0.00996939968358619, '2401.TW': 0.01672010233042492, '2408.TW': 0.0008059644416253733, '2434.TW': 0.02379222774215231, '2436.TW': 0.007729389263095365, '2441.TW': 0.007998070922411938, '2449.TW': 0.06400115158253969, '2451.TW': 0.053574533602239995, '2454.TW': 0.054367712003637284, '2458.TW': 0.05262099239335119, '2481.TW': 0.017326260943278004, '3006.TW': 0.0034870617973606567, '3014.TW': 0.03897160363653521, '3016.TW': 0.027696095547469705, '3034.TW': 0.05510410430383609, '3035.TW': 0.009677036083580769, '3041.TW': 0.0011423622028232995}
最佳投資組合的年化回

In [121]:
print(len(stock_names),len(tickers))

30 30
